## 异构图学习(heterogeneous graph learning)

- **同构图**：节点和边的类型均只有一种，比如社交网络中只存在一种节点类型（用户节点）和一种边类型（用户-用户的连边）。

- **异构图**：节点的类型或者边的类型有多种，比如论文引用网络，节点有两种类型（作者节点和文章节点），边的类型有作者-作者间的共同创作关系，作者-文章间的从属关系，文章-文章间的引用关系。

- **属性图**：图的节点上存在初始属性，可以用作后续节点的特征。

- **动态图**：图中的节点或者边都是随着时间变化的，可能增加也可能减少，每个时刻用一个图表示。

- **关系图**：图表示了一种节点之间的隐含关系，比如知识图谱。

## 示例图

&emsp;&emsp;看一下来自[OGB](https://ogb.stanford.edu/)数据集的异构[ogbn-mag](https://ogb.stanford.edu/docs/nodeprop/)网络：

<img src="../../images/12-hg_example.svg" width="60%">

In [5]:
node_nums = 59965 + 736389 + 1134649 + 8740
edge_nums = 7505078 + 5416271 + 7145660 + 1043998
print("node_nums\n", node_nums)
print('edge_nums\n', edge_nums)

node_nums
 1939743
edge_nums
 21111007


&emsp;&emsp;给定的异构图有 1,939,743 个节点，分为作者、论文、机构和研究领域四种节点类型。

&emsp;&emsp;它还有 21,111,007 条边，它们也是以下四种类型之一：

- writes：作者写了一篇特定的论文

- 隶属于：作者隶属于特定机构

- cites：一篇论文引用了另一篇论文

- has topic : 一篇论文有一个特定研究领域的主题

&emsp;&emsp;该图的任务是根据存储在图中的信息推断每篇论文所在的场所venue（会议或者期刊），在ogbn-mag中总共有349个不同的venue，使任务成为349类的分类问题；

## 创建异构图

&emsp;&emsp;我们可以创建一个类型`torch_geometric.data.HeteroData`的数据对象，为每个类型分别定义节点特征张量、边缘索引张量和边缘特征张量。

```python
data = HeteroData()

data['paper'].x = ... # [num_papers, num_features_paper]
data['author'].x = ... # [num_authors, num_features_author]
data['institution'].x = ... # [num_institutions, num_features_institution]
data['field_of_study'].x = ... # [num_field, num_features_field]

data['paper', 'cites', 'paper'].edge_index = ... # [2, num_edges_cites]
data['author', 'writes', 'paper'].edge_index = ... # [2, num_edges_writes]
data['author', 'affiliated_with', 'institution'].edge_index = ... # [2, num_edges_affiliated]
data['author', 'has_topic', 'institution'].edge_index = ... # [2, num_edges_topic]

data['paper', 'cites', 'paper'].edge_attr = ... # [num_edges_cites, num_features_cites]
data['author', 'writes', 'paper'].edge_attr = ... # [num_edges_writes, num_features_writes]
data['author', 'affiliated_with', 'institution'].edge_attr = ... # [num_edges_affiliated, num_features_affiliated]
data['paper', 'has_topic', 'field_of_study'].edge_attr = ... # [num_edges_topic, num_features_topic]
```

&emsp;&emsp;节点或边张量将在第一次访问时自动创建并由字符串键索引。节点类型由单个字符串标识，而边类型使用三元组字符串标识：边类型标识符和边类型可以存在的两个节点类型。因此，数据对象允许每种类型具有不同的特征维度。(source_node_type, edge_type, destination_node_type)。

&emsp;&emsp;包含按属性名称而不是按节点或边类型分组的异构信息的字典可以直接访问，`data.{attribute_name}_dict`并在以后作为`GNN`模型的输入：

```python
model = HeteroGNN(...)

output = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)
```

In [7]:
from torch_geometric.datasets import OGB_MAG

dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
data = dataset[0]

ImportError: cannot import name 'OGB_MAG' from 'torch_geometric.datasets' (/Users/hezhiqiang01/Desktop/anaconda/anaconda3/envs/ecole/lib/python3.9/site-packages/torch_geometric/datasets/__init__.py)